## CMS Exclusions Dataset Exploratory Data Analysis

In [ ]:
# Import necessary libraries such as numpy and pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# This function will provide basic insights into any data
def quick_summary(df):
    print("=== Missing Values ===")
    print(df.isnull().sum())
    print("\n=== Basic Info ===")
    print(df.info())
    print("\n=== Sample Rows ===")
    print(df.head())

### OIG Exclusions Data

In [ ]:
# Load the OIG exclusions file into a pandas dataframe
exclusions_file = 'cms_data/leie.csv'
exclusions = pd.read_csv(exclusions_file)
exclusions = exclusions[exclusions.NPI > 0].set_index('NPI')

In [ ]:
quick_summary(exclusions)

In [ ]:
# Drops unimportant columns or columns with mostly missing values
exclusions = exclusions.drop(
    columns=['UPIN', 'MIDNAME', 'WVRSTATE', 'BUSNAME', 'REINDATE', 'WAIVERDATE'],
    errors='ignore'  
)

exclusions.head()

In [ ]:
# Drops rows with N/A values in specialty and 'dob' column as that is important for insights
exclusions = exclusions.dropna(subset=['SPECIALTY'])
exclusions = exclusions.dropna(subset=['DOB'])
print(exclusions.shape)

In [ ]:
# Changes dates from YYYYMMDD to YYYY-MM-DD
date_cols = ['DOB', 'EXCLDATE']
for col in date_cols:
    exclusions[col] = exclusions[col].astype(str).str.replace(r'\.0$', '', regex=True)
    exclusions[col] = exclusions[col].replace(['nan', 'NaN', '', '00000000'], np.nan)
    exclusions[col] = pd.to_datetime(exclusions[col], format='%Y%m%d', errors='coerce')

exclusions.head()

In [ ]:
# Creates two new columns: Age of the provider at exclusion, and the year excluded
exclusions['age_at_exclusion'] = (
    (exclusions['EXCLDATE'] - exclusions['DOB']).dt.days / 365.25
)
exclusions['year_excl'] = exclusions['EXCLDATE'].dt.year

In [ ]:
excltype_map = {
    '1128a1': 'Conviction – Medicare Fraud',
    '1128a2': 'Patient Abuse/Neglect',
    '1128a3': 'Felony – Drugs',
    '1128a4': 'Felony – Healthcare Fraud',
    '1128b1': 'Misdemeanor – Fraud',
    '1128b2': 'Default on Student Loan',
    '1128b3': 'License Revocation',
    '1128b4': 'Unlawful Claims',
    '1128b5': 'Kickbacks/Bribery',
    '1128b6': 'False Claims',
    '1128b7': 'Obstruction of Audit',
    '1128b8': 'Controlled Substances Violation',
    '1128b9': 'Insurance Fraud',
    '1128b10': 'Unlawful Billing',
    '1128b11': 'Quality of Care Violation',
    '1128b12': 'Civil Monetary Penalty',
    '1128b13': 'False Statement',
    '1128b14': 'Suspension/Exclusion',
    '1128b15': 'License Suspension',
    '1128b16': 'Federal Program Violation',
}

# Apply mapping
exclusions['EXCLTYPE'] = exclusions['EXCLTYPE'].replace(excltype_map)

# Optional: Fill unmapped types with 'Other' or 'Unknown'
exclusions['EXCLTYPE'] = exclusions['EXCLTYPE'].fillna('Other')

# Check result
print(exclusions['EXCLTYPE'].value_counts())

In [ ]:
quick_summary(exclusions)

In [ ]:
exclusions['SPECIALTY'].value_counts().head(15)

In [ ]:
plt.figure(figsize=(10,5))
exclusions['STATE'].value_counts().head(15).plot(kind='bar')
plt.title('Top 15 States by Number of Exclusions')
plt.xlabel('State')
plt.ylabel('Number of Exclusions')
plt.show()

In [ ]:
sns.countplot(y='EXCLTYPE', data=exclusions, order=exclusions['EXCLTYPE'].value_counts().index)
plt.title('Most Common Exclusion Types')
plt.show()

In [ ]:
sns.set(style="whitegrid")

# ---- Age Distribution ----
plt.figure(figsize=(6, 4))
sns.histplot(exclusions['age_at_exclusion'].dropna(), kde=True, bins=20)
plt.title("Age at Exclusion Distribution")
plt.xlabel("Age (years)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# ---- Temporal Trends ----
plt.figure(figsize=(10, 5))
sns.countplot(
    data=exclusions,
    x='year_excl',
    order=sorted(exclusions['year_excl'].dropna().unique())
)
plt.title("Exclusions Over Time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
top_states = (
    exclusions['STATE']
    .value_counts()
    .nlargest(10)
    .reset_index()
)

# Rename columns clearly
top_states.columns = ['STATE', 'COUNT']

# --- Visualization ---
plt.figure(figsize=(8, 5))
sns.barplot(data=top_states, x='STATE', y='COUNT', palette='viridis')

plt.title("Top 10 States by Medicare Exclusions", fontsize=14, weight='bold')
plt.xlabel("State")
plt.ylabel("Number of Exclusions")

# Add labels above bars
for i, row in top_states.iterrows():
    plt.text(i, row['COUNT'] + 5, int(row['COUNT']), ha='center')

plt.tight_layout()
plt.show()

### CMS Medicare Provider Data

In [ ]:
files = os.listdir('cms_data')
files

In [ ]:
fil = files[6]
data = pd.read_csv('cms_data/'+fil)

In [ ]:
quick_summary(data)

In [ ]:
plt.figure(figsize=(12, 7))

# Taking a sample of the data can make the plot less crowded and faster to render
sample_df = data.sample(n=5000, random_state=42)

ax = sns.scatterplot(
    data=sample_df,
    x='Bene_Avg_Risk_Scre',
    y='Tot_Mdcr_Pymt_Amt',
    hue='Tot_Srvcs',
    size='Tot_Srvcs', # Vary point size by number of services
    sizes=(20, 200),
    palette='viridis',
    alpha=0.6
)

ax.set_yscale('log')
plt.title('Patient Risk Score vs. Total Medicare Payment', fontsize=16)
plt.xlabel('Beneficiary Average Risk Score', fontsize=12)
plt.ylabel('Total Medicare Payment Amount (Log Scale)', fontsize=12)
plt.legend(title='Total Services')
plt.tight_layout()
plt.style.use("seaborn-v0_8-dark-palette")
plt.show()